In [1]:
import xml.etree.ElementTree as ET
import re
import csv

def tokenize(text):
    # Tokeniza el texto en menciones, hashtags, palabras y otros caracteres
    tokens = re.findall(r'(@\w+|#\w+|\w+|[^\w\s])', text, re.UNICODE)
    # Filtra tokens vacíos que puedan aparecer por espacios en blanco
    return [t for t in tokens if t.strip()]

def process_tweet(tweet_element):
    parts = []
    # Procesa el texto inicial del tweet
    current_text = tweet_element.text.strip() if tweet_element.text else ''
    if current_text:
        parts.append((current_text, None))
    # Procesa cada elemento hijo (etiquetas sentiment)
    for child in tweet_element:
        aspect = child.get('aspect')
        # Procesa el texto dentro de la etiqueta sentiment
        child_text = child.text.strip() if child.text else ''
        if child_text:
            parts.append((child_text, aspect))
        # Procesa el texto después de la etiqueta sentiment (tail)
        tail_text = child.tail.strip() if child.tail else ''
        if tail_text:
            parts.append((tail_text, None))
    # Genera tokens y etiquetas BIO para cada parte
    tokens = []
    labels = []
    for part_text, aspect in parts:
        part_tokens = tokenize(part_text)
        if not part_tokens:
            continue
        if aspect is None:
            part_labels = ['O'] * len(part_tokens)
        else:
            part_labels = ['B-' + aspect] + ['I-' + aspect] * (len(part_tokens) - 1)
        tokens.extend(part_tokens)
        labels.extend(part_labels)
    return tokens, labels

# Parsear el archivo XML
tree = ET.parse('../Data/stompol-test-tagged.xml')
root = tree.getroot()

tweets = root.findall('tweet')

# Escribir el archivo CSV
with open('tokens_labels.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    writer.writerow(['tokens', 'labels'])
    for tweet in tweets:
        tokens, labels = process_tweet(tweet)
        writer.writerow([str(tokens), str(labels)])

In [2]:
import pandas as pd
import ast

def transform_labels_o_to_zero(csv_path):
    # Leer el CSV con separador ;
    df = pd.read_csv(csv_path, sep=';')
    
    # Función para convertir los 'O' en 0
    def convert_o_to_zero(label_list):
        # Convertir la cadena de texto a lista
        try:
            labels = ast.literal_eval(label_list)
        except:
            labels = label_list
            
        # Reemplazar 'O' por 0 manteniendo otros valores
        return [0 if label == 'O' else label for label in labels]
    
    # Aplicar la conversión a toda la columna labels
    df['labels'] = df['labels'].apply(convert_o_to_zero)
    
    # Guardar el nuevo CSV
    df.to_csv(csv_path, sep=';', index=False)
    print(f"Archivo transformado guardado en: {csv_path}")

# Ejemplo de uso
transform_labels_o_to_zero("../Preprocesing/tokens_labels.csv")

Archivo transformado guardado en: ../Preprocesing/tokens_labels.csv


In [13]:
import pandas as pd
import ast

def transform_labels_o_to_zero(csv_path):
    # Leer el CSV con separador ;
    df = pd.read_csv(csv_path, sep=';')
    
    # Función para convertir los 'O' en 0
    def convert_o_to_zero(label_list):
        # Convertir la cadena de texto a lista
        try:
            labels = ast.literal_eval(label_list)
        except:
            labels = label_list
            
        # Reemplazar 'O' por 0 manteniendo otros valores
        return [5 if label == 'I-Otros_aspectos' else label for label in labels]
    
    # Aplicar la conversión a toda la columna labels
    df['labels'] = df['labels'].apply(convert_o_to_zero)
    
    # Guardar el nuevo CSV
    df.to_csv(csv_path, sep=';', index=False)
    print(f"Archivo transformado guardado en: {csv_path}")

# Ejemplo de uso
transform_labels_o_to_zero('tokens_labels.csv')

Archivo transformado guardado en: tokens_labels.csv
